In [ ]:
import boto3

# Client

In [ ]:
s3 = boto3.client("s3",
                  region_name = 'us-east-1',
                  aws_access_key_id = AWS_KEY_ID,
                  aws_secret_access_key = AWS_SECRET)

In [ ]:
Permissions summary

Name                           Type           Used as
AmazonS3FullAccess             AWS managed    Permissions policy
AmazonSNSFullAccess            AWS managed    Permissions policy
AmazonRekognitionFullAccess    AWS managed    Permissions policy
ComprehendFullAccess           AWS managed    Permissions policy
IAMUserChangePassword          AWS managed    Permissions policy

# S3 - Simple Storage Service

In [ ]:
# Create the buckets
response_staging = s3.create_bucket(Bucket='gim-staging')
response_processed = s3.create_bucket(Bucket='gim-processed')
response_test = s3.create_bucket(Bucket='gim-test')

# Print out the response
print(response_staging)

#listing buckets
bucket_response = s3.list_buckets() 
buckets = bucket_response['Buckets']     #Returns dictionary

#Delete the buckets
s3.delete_bucket(Bucket='name')

# Delete all the buckets with 'gim', create replacements.
for bucket in response['Buckets']:
  if 'gim' in bucket['Name']:
      s3.delete_bucket(Bucket=bucket['Name'])
    
s3.create_bucket(Bucket='gid-staging')
s3.create_bucket(Bucket='gid-processed')
  
# Print bucket listing after deletion
response = s3.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'])
    
#deleting objects
s3.delete_object(Bucket = 'gid-requests',
                Key = 'gid_requests_2018_12_30.csv')

    
#uploading files
s3.upload_file(
    Filename = 'gid_requests_2019_01_01.csv',   #add filepath
    Bucket = 'gid-requests',
    key = 'gid_requests_2019_01_01.csv'
    ExtraArgs={'ACL':'public-read'})

#listing objects in a bucket
response = s3.list_objects(Bucket = 'gid-requests',
                           MaxKeys = 2,
                           Prefix = 'gid_requests_2019_')
print(response)

#getting object metadata
response = s3.head_objects(Bucket = 'gid-requests',
            Key = 'gid_requests_2018_12_30.csv')
print(response)

# Iterate over the objects
if 'Contents' in response:
  for obj in response['Contents']:
      # Delete the object
      s3.delete_object(Bucket='gid-staging', Key=obj['Key'])
    
# Print the keys of remaining objects in the bucket
response = s3.list_objects(Bucket='gid-staging')

for obj in response['Contents']:
  	print(obj['Key'])
    

#downloadinng the file
s3.download_file(Filename='gid_requests_downed.csv', #specify where to download (filepath)
                Bucket = 'gid-requests',
                Key = 'gid_requests_2018_12_30.csv')



In [ ]:
#Security

#ACL (Access controll lists)
    #set ACL to 'public-read' by default it will be private
    s3.put_object_acl(Bucket='gid-requests', Key='potholes.csv', ACL='public-read')
    
#Accessing public objects
    #s3 object URL template
    https://{bucket},{key} #ex-  https://gid-requests.2019/potholes.csv  URL for key='2019/potholes.csv'
    #The Public Object URL format is 'https://{bucket}.s3.amazonaws.com/{key}'.  
    
    #generating public urls
    url = 'https://{},{}.format("gid-requests", "2019/potholes.csv")'
    df = pd.read_csv(url) #works if access is public
    
#Accessing private objects
    #Downloading a priivate file
    s3.download_file(Filename = 'potholes_local.csv',
                    Bucket = 'gid-staging',
                    Key = '2019/potholes_private.csv')
    
    df = pd.read_csv('https://gid-staging.potholes.csv') # read directly or locally
    
    obj = s3.get_object(Bucket='gid-requests', Key='2019/potholes.csv')
    print(obj) # similar response as head_object with extra body key
    #read StreamingBody in pandas
    pd.read_csv(obj['Body'])
    
#Generating presigned URL
share_url = s3.generate_presigned_url(ClientMethod='get-object',
                                     ExpiresIn=3600,   # access expires in 1hr
                                     Params={'Bucket': 'gid-requests', 'Key':'potholes.csv'})
    
pd.read_csv(share_url) #read in pandas

#Loading multiple files in the DataFrame
df_list = [] #list to hold dataframes

#Request the list of csv's from s3 with prefix ; get contents
response = s3.list_objects(Bucket = 'gid-requests', Prefix = '2019/')

#get response contents
request_files = response['contents']

#iterate over each object 
for file in request_files:
    obj = s3.get_object(Bucket='gid-requests', Key=file[Key])
        
    #read it as dataframe
    obj_df = pd.read_csv(obj['Body'])
    #Append dataframe to list
    df_list.append(obj_df)
    
#Concatenate all the dataframes in the list
df = pd.concat(df_list)

#Preview the dataframe
df.head()

#dataframe to html
df.to_html('table_agg.html',
          render_links = True, #all the links are clickable
          columns['service_name', 'request_count', 'info_link'] # selects on these columns
          border = 0) # 0 will remove border and 1 will add border

#Uploading a HTML file in s3
s3.upload_file(Filename = './table_agg.html',
              Bucket='datacamp-website',
              Key='table.html',
              ExtraArgs=['ContentType' : 'text/html', #loads the file as HTML
                         'ACL' : 'public-read'])
    #file can be accesses on https://datacamp-website.table.html
    
#generating an index page
# List he gid-reports bucket objects starting with 2019/
r = s3.list_objects(Bucket = 'gid-reports', Prefix = '2019/')

#Convert the response contents to Dataframe
objects_df = pd.DataFrame(r['Contents'])

#Create a column "Link" that contains website url + key
base_url = "https://datacamp-website"
objects_df['Link'] = base_url + objects_df['Key']

# Write the dataframe to html
objects_df.to_html('report_listing.html',
                  columns = ['Link', 'LastModified', 'Size'],
                  render_links = True)
#upload html as above  

# SNS - Simple Notification Service


In [ ]:
#SNS Simple notification service
sns = boto3.client("sns",
                  region_name = 'us-east-1',
                  aws_access_key_id = AWS_KEY_ID,
                  aws_secret_access_key = AWS_SECRET)

#Topics
response = sns.create_topic(Name='city_alerts')['TopicArn']

#listing topics
response = sns.list_topics()

sns.delete_topic(TopicArn='arn:aws:sns:us-east-1:32022756131:city_alerts')
# Get the current list of topics
topics = sns.list_topics()['Topics']

for topic in topics:
  # For each topic, if it is not marked critical, delete it
  if "critical" not in topic['TopicArn']:
    sns.delete_topic(TopicArn=topic['TopicArn'])
    
# Print the list of remaining critical topics
print(sns.list_topics()['Topics'])

#Subscriptions
#creating sms subscriptiiions
response = sns.subscribe(TopicArn = 'arn:aws:sns:us-east-1:32022756131:city_alerts'
                        Protocol = 'SMS',
                        Endpoint = '+917893009903') #phone numbers are automatically confirmed
#creating email subscriptiiions
response = sns.subscribe(TopicArn = 'arn:aws:sns:us-east-1:32022756131:city_alerts'
                        Protocol = 'email',
                        Endpoint = 'lohithdevulapalli@gmail.com') #verify on email to confirm # SubscriptionArn when verified untill then SubscriptionArn shows 'pending confirmation'
#listing subscriptions
sns.list_subscriptions()['Subscriptions']
#delete subscription
sns.unsubscribe(SubscriptionArn='arn:aws:sns:us-east-1:32022756131:city_alerts:9f2dad1d-8844-4fe8')
#delete multiple subscriptions
    #get a list of subscriptions
    response = list_subscriptions_by_topic(TopicArn = 'arn:aws:sns:us-east-1:32022756131:city_alerts')
    subs = response['Subscriptions']
    
    #unsubscribe sms subscriptions
    for sub in subs:
        if sub['protocol'] == 'sms':
            sns.unsubscribe(sub['SubscriptionsArn'])

In [ ]:
#sending messages
#publishing to a topic
num_of_reports = 137
response = sns.publish(TopicArn = 'arn:aws:sns:us-east-1:32022756131:city_alerts',
                      Message = 'THere are {} reports outstanding'.format(num_of_reports),
                      Subject = 'Subject line for email') # subject line will not be visible to for SMS
#send a single sms
response = sns.publish(PhoneNumber = '+917893009903',
                      Message = 'Body text of SMS or e-mail')

In [ ]:
#building a notification system
sns = boto3.client("sns",
                  region_name = 'us-east-1',
                  aws_access_key_id = AWS_KEY_ID,
                  aws_secret_access_key = AWS_SECRET)
    #create topic and store their arns
    trash_arn = sns.create_topic(Name = 'trash_notifications')['TopicArn']
    streets_arn = sns.create_topic(Name='streets_notificatiions')['TopicArn']
    
    #subscribing users to the topic
    contacts = pd.read_csv('http://gir-staging.contacts.csv')
    
    #example csv
    ?Name             Email              Phone         Department
    John Smith        js@fake.com        +11224567890  trash
    Fanny Mae         fannyma3@fake.com  +11234597890  trash
    Janessa Goldsmith whoami@fake.com    +11534567890  streets
    Evelyn Monroe     Evely@fake.com     +11234067890  streets
    
    #create subscribe_user method
    def subscribe_user(user_row):
        if user_row['Department'] == 'trash':
            sns.subscribe(TopicArn = trash_arn, Protocol = 'sms', EndPoint=str(user_row['Phone']))
            sns.subscribe(TopicArn = trash_arn, Protocol = 'email', EndPoint=str(user_row['Email']))
        else:
            sns.subscribe(TopicArn = streets_arn, Protocol = 'sms', EndPoint=str(user_row['Phone']))
            sns.subscribe(TopicArn = streets_arn, Protocol = 'email', EndPoint=str(user_row['Email']))
            
    #Apply subscribe_user method to every row
    contacts.apply(subscribe_user, axis=1)
    
    #get the aggregated  numbers 
    #load January report into a dataframe
    df = pd.read_csv('http://gid-reports.2019/feb/final_report.csv')
    
    #sample report
    service_name                  count
    Illegal Dumping               2580
    Potential Missed Collection   150
    Pothole                       1170
    Traffic Sign - Maintain       210
    Traffic Signal Head Turned    60
    Traffic Signal Light Out      120
    
    #set the index so we can access counts by service name directly
    df.set_index('service_name', inplace=True)
    
    #get the aggregated number
    trash_violations_count = df.at['Illegal Dumping', 'count']
    streets_violations_count = df.at['Pothole', 'count']
    
    #send alerts
    if trash_violations_count > 100:
        #Construct the  message to send
        message = "Trash violations count is now {}".format(trash_violations_count)
        #send message
        sns.publish(TopicArn=trash_arn,
                   Message = message,
                   Subject = "Trash Alert")
        
    if streets_violations_count > 30:
        #Construct the  message to send
        message = "Streets violations count is now {}".format(streets_violations_count)
        #send message
        sns.publish(TopicArn=streets_arn,
                   Message = message,
                   Subject = "Streets Alert")    

# Rekognition

In [ ]:
#Rekognition
#Initialize s3 client
s3 = boto3.client("s3",
                  region_name = 'us-east-1',
                  aws_access_key_id = AWS_KEY_ID,
                  aws_secret_access_key = AWS_SECRET)
#Upload an image to s3
s3.upload_file(Filename = 'report.jpg', Key='report.jpg', Bucket='datacamp-img')

#Initialize Rekog client
rekog = boto3.client('rekognition', region_name='us-east-1', 
                     aws_access_key_id = AWS_KEY_ID,
                     aws_secret_access_key = AWS_SECRET)

#Detect labels in Image
response = rekog.detect_labels(Image={'S3Object':{'Bucket': 'datacamp-img', 
                                                  'Name': 'report.jpg'}},
                               MaxLabels=10,
                               MinConfidence=95)

#Detect text
response = rekog.detect_text(Image={'S3Object':{'Bucket': 'datacamp-img', 
                                                  'Name': 'report.jpg'}})

# Count of multiple cat objects
# Create an empty counter variable
cats_count = 0
# Iterate over the labels in the response
for label in response['Labels']:
    # Find the cat label, look over the detected instances
    if label['Name'] == 'Cat':
        for instance in label['Instances']:
            # Only count instances with confidence > 85
            if (instance['Confidence'] > 85):
                cats_count += 1
# Print count of cats
print(cats_count)

# Create empty list of words
words = []
# Iterate over the TextDetections in the response dictionary
for text_detection in response['TextDetections']:
  	# If TextDetection type is WORD, append it to words list
    if text_detection['Type'] == 'WORD':
        # Append the detected text
        words.append(text_detection['DetectedText'])
# Print out the words list
print(words)  #prints     ['NO', 'PARKING', '7', 'AM', 'TO', '12', 'NOON', 'MONDAY']

# Create empty list of lines
lines = []
# Iterate over the TextDetections in the response dictionary
for text_detection in response['TextDetections']:
  	# If TextDetection type is Line, append it to lines list
    if text_detection['Type'] == 'LINE':
        # Append the detected text
        lines.append(text_detection['DetectedText'])
# Print out the words list
print(lines) #prints     ['NO PARKING', '7 AM', 'TO', '12 NOON', 'MONDAY']

# Sentiment Analysis

In [ ]:
#comprehending text
#initialize client
translate = boto3.client('translate',
                        region_name = 'us-east-1',
                        aws_access_key_id = AWS_KEY_ID,
                        aws_secret_access_key = AWS_SECRET)

#Translating text
response = translate.translate_text(Text='Hello, how are you?',
                                   SourceLanguageCode = 'auto', # Uses aws comprehend automatically to detect as auto
                                   TargetLanguageCode = 'es') # es refers to spanish

#Detecting text
comprehend = boto3.client('comprehend',
                         region_name = 'us-east-1',
                         aws_access_key_id = AWS_KEY_ID,
                         aws_secret_access_key = AWS_SECRET)
#detect dominant language
response = comprehend.detect_dominant_language(Text='Hay basura por todas partes a lo largo de la carretera')

#detect text sentiment
response = comprehend.detect_sentiment(Text = 'Datacamp students are amazing!',
                                      LanguageCode='en')
#understanding sentiment(one liner)
response = comprehend.detect_sentiment(Text = 'Datacamp students are amazing!',
                                      LanguageCode='en')['Sentiment']

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:*",
                "s3-object-lambda:*"
            ],
            "Resource": "*"
        }
    ]
}